In [229]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import cufflinks as cf

N=60
N_next=50

def generate_random_numbers(A_ub,B_ub,bounds):
    n=len(bounds)
    flag=False
    res=[]
    while(flag==False):
        res=[]
        flag=True
        for i in range(n):
            res.append(np.random.rand()*(bounds[i][1]-bounds[i][0])+bounds[i][0])
        B=A_ub@np.array(res)
        for i in (B>B_ub):
            if(i):
                flag=False
    return np.array(res)
A_ub=np.array([[-9, -1], [-9, 1]])
B_ub=np.array([-6, -1])
bounds=np.array([(0.1, 1), (0, 5)])
# 生成满足约束条件的随机数
result = generate_random_numbers(A_ub,B_ub,[(0.1, 1), (0, 5)])
result

array([0.73378864, 4.41309261])

In [230]:

para=([[-9, -1], [-9, 1]],[-6, -1],[(0.1, 1), (0, 5)])

class point:
    def __init__(self,pos) -> None:
        self.dominates=[]
        self.domination_count=0
        self.pos=pos
        self.f=np.array([pos[0],(1+pos[1])/pos[0]])
        self.dis=0
        
    def __str__(self):
        return self.pos
        
P=[point(generate_random_numbers(*para)) for i in range(N)]
[p.pos for p in P]

[array([0.99419983, 4.05905566]),
 array([0.93685346, 2.37938928]),
 array([0.59681777, 2.99595299]),
 array([0.94499254, 1.02549725]),
 array([0.70496198, 1.51411811]),
 array([0.50169718, 2.13617069]),
 array([0.62679522, 2.07683045]),
 array([0.8118449 , 3.33140029]),
 array([0.67902955, 3.64341943]),
 array([0.84431748, 2.54706647]),
 array([0.74619023, 3.25052123]),
 array([0.42393812, 2.36246702]),
 array([0.95665451, 3.20894194]),
 array([0.83302254, 1.87420209]),
 array([0.86796145, 4.03471383]),
 array([0.85489044, 4.77524203]),
 array([0.97166519, 4.76143515]),
 array([0.58028744, 3.80756617]),
 array([0.94756527, 1.16540554]),
 array([0.40842821, 2.6752001 ]),
 array([0.71698485, 1.95717681]),
 array([0.7052628, 0.719416 ]),
 array([0.59318315, 0.83700852]),
 array([0.652055  , 0.71782632]),
 array([0.54676721, 3.16560217]),
 array([0.67717979, 3.09234193]),
 array([0.80524118, 3.32029104]),
 array([0.99243743, 0.15500787]),
 array([0.8052213 , 3.55281174]),
 array([0.444337

In [231]:
import plotly.graph_objects as go
class objects:
    def __init__(self,data) -> None:
        self.data=data
        self.f=np.array([p.f for p in data] )
    
    def plot(self):
        fig=go.Figure()
        fig.add_trace(go.Scatter(x=self.f[:,0],y=self.f[:,1],mode="markers")) 
        fig.show()
objects(P).plot()


In [232]:
#Non-dominated Sorting
import itertools
def init(P):
    for p in P:
        p.dominates=[]
        p.domination_count=0
        p.dis=0
    for ps in itertools.combinations(P,2):
        if(all(ps[0].f<=ps[1].f)):
            ps[0].dominates.append(ps[1])
            ps[1].domination_count+=1
        elif(all(ps[1].f<=ps[0].f)):
            ps[1].dominates.append(ps[0])
            ps[0].domination_count+=1
    
init(P)
P[10].pos,P[10].domination_count

(array([0.74619023, 3.25052123]), 11)

In [233]:
import copy

def non_dominated_sort(P):
    P_cp=copy.deepcopy(P)
    ans=[]
    while(len(P_cp)!=0):
        fonts=[]
        for p in P_cp:
            if(p.domination_count==0):
                fonts.append(p)
                P_cp.remove(p)
        for font in fonts:
            for pd in font.dominates:
                    pd.domination_count-=1
        ans.append(fonts)
    return ans

sorted_P=non_dominated_sort(P)
sorted_P[0]


In [234]:
objects(sorted_P[0]).plot()

In [235]:
#Crowding Distance Sorting
# 评价多样性
INF=10000

def crowding_dist_sort(P):
    for i in range(2):# 每个目标
        Pi=sorted(P,key=lambda s:s.f[i])
        values=[p.f[i]for p in Pi]
        fmax=max(values)
        fmin=min(values)
        df=fmax-fmin
        Pi[0].dis=INF
        Pi[-1].dis=INF
        for i in range(1,len(Pi)-1):
            Pi[i].dis+=(values[i+1]-values[i-1])/df
        res=sorted(P,key=lambda s:s.dis,reverse=True)
        return res
ans=crowding_dist_sort(sorted_P[0])
for p in sorted_P[0]:
    print(f"{p.pos} {p.dis}")
[p.dis for p in ans]

[0.50169718 2.13617069] 0.21437693368799057
[0.42393812 2.36246702] 0.15970461696889016
[0.40842821 2.6752001 ] 10000
[0.7052628 0.719416 ] 0.26188059556084575
[0.652055   0.71782632] 0.2673358042475445
[0.99243743 0.15500787] 10000
[0.54913622 1.87474364] 0.25745796234777246
[0.80499568 0.43768427] 0.30120891090568574
[0.88117158 0.35597205] 0.32095682512249163


[10000,
 10000,
 0.32095682512249163,
 0.30120891090568574,
 0.2673358042475445,
 0.26188059556084575,
 0.25745796234777246,
 0.21437693368799057,
 0.15970461696889016]

In [236]:

def generate_next_P(N_next,sorted_P):
    P=[]
    cur=0
    for font in sorted_P:
        cur+=len(font)
        if(N_next-cur<0):
            ans=crowding_dist_sort(font)
            P.extend(ans[:N_next-len(P)])
            return P
        elif(N_next==cur):
            P.extend(font)
            return P
        else:
            P.extend(font)

Ptt=generate_next_P(N_next,sorted_P)
objects(Ptt).plot()

In [237]:
precent=0.2

def apply_mutation(Ptt):
    num=N-len(Ptt)
    for i in range(num):
        tmp=np.random.choice(Ptt)
        pos=tmp.pos
        pos+=2*(np.random.rand()-0.5)*precent*pos
        flag=True
        count=0
        while(flag):
            if(all(A_ub@pos<=B_ub)and bounds[0][0]<pos[0]<bounds[0][1] and bounds[1][0]<pos[1]<bounds[1][1]):
                child=point(pos)
                Ptt.append(child)
                flag=False
            elif(count>100):
                child=copy.deepcopy(np.random.choice(Ptt))
                Ptt.append(child)
                flag=False
            else:
                tmp=np.random.choice(Ptt)
                pos=tmp.pos
                pos+=2*(np.random.rand()-0.5)*precent*pos
                count+=1
    return Ptt
objects(apply_mutation(Ptt)).plot()

In [238]:
rounds=50
for i in range(rounds):
    init(Ptt)
    sorted_P=non_dominated_sort(Ptt)
    Ptt=generate_next_P(N_next,sorted_P)
    Ptt=apply_mutation(Ptt)
objects(Ptt).plot()

In [239]:
len(Ptt)

60